# Итерируемся правильно: 20 приёмов использования в Python модуля `itertools`

Материал подготовлен для [Библиотеки программиста](https://proglib.io/).

В декабре 2019 года мы подробно рассказали [о модуле collections](https://proglib.io/p/ne-izobretat-velosiped-ili-obzor-modulya-collections-v-python-2019-12-15). Другой популярный компонент стандартной библиотеки – [itertiools](https://docs.python.org/3/library/itertools.html).

Модуль itertools содержит строительные блоки итераторов, основанные на конструкциях из языков программирования [APL](https://ru.wikipedia.org/wiki/APL_(%D1%8F%D0%B7%D1%8B%D0%BA_%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D1%8F)), [Haskell](https://ru.wikipedia.org/wiki/Haskell) и [SML](https://ru.wikipedia.org/wiki/Standard_ML). Ниже мы опишем набор быстрых и эффективных в отношении памяти инструментов, полезных как самостоятельно, так и в сочетании. Вместе они образуют «алгебру итераторов» для программ на чистом Python.

**Цель публикации** – в сжатой форме рассмотреть распространённые примеры и шаблоны использования модуля itertools.

Начнем с импорта:

In [1]:
import itertools

Чтобы лучше запомнить функции модуля, мы не станем использовать конструкцию `from ... import *`, а будем обращаться к методам модуля через его имя.

# 1. Бесконечный счётчик

Функция `itertools.count(start=0, step=1)` создаёт бесконечный итератор. Можно задать начальное значение и шаг итерирования.

In [2]:
cnt = itertools.count(start=2020, step=4)
next(cnt)

2020

In [3]:
next(cnt)

2024

In [4]:
next(cnt)

2028

Пример использования функции:

In [5]:
days = [366]*4
list(zip(itertools.count(2020, 4), days))

[(2020, 366), (2024, 366), (2028, 366), (2032, 366)]

Чтобы продолжить счёт при прерывании выполнения программы передайте последнее значение новому объекту итератора в виде параметра `start`.

Если необходимо подсчитывать число вхождений элементов в список или кортеж, обратите внимание на `Counter()` из [модуля collections](https://proglib.io/p/ne-izobretat-velosiped-ili-obzor-modulya-collections-v-python-2019-12-15).

# 2. Упаковка по более длинной последовательности
Если последовательности имеют неодинаковую длину, `zip()` ограничивается самой короткой:

In [6]:
list(zip(range(0, 10), range(0, 5)))

[(0, 0), (1, 1), (2, 2), (3, 3), (4, 4)]

Но такое сокращение приводит к потере информации. Чтобы сохранить обе последовательности, используйте `itertools.zip_longest()`:

In [7]:
for (i, j) in itertools.zip_longest(range(0, 10), range(0, 5)):
    print(i, j)

0 0
1 1
2 2
3 3
4 4
5 None
6 None
7 None
8 None
9 None


Вместо `None` функция может подставлять значение, переданное аргументу `fillvalue`.

# 3. Итератор с нарастающим итогом
Суммирование нарастающим (накопительным) итогом – вид сложения последовательности чисел. Например, так считается квартальная прибыль Каждый элемент складывается с суммой всех предшествовавших элементов. В следующем примере 1 и 2 даёт 3, сумма 1, 2 и 3 равна 6 и т. д. Описанный тип работы с последовательностью воплощен в `itertools.accumulate(iterable, func=operator.add, *, initial=None)`:

In [8]:
list(itertools.accumulate(range(1, 10)))

[1, 3, 6, 10, 15, 21, 28, 36, 45]

Чтобы вывести данные, мы используем список. При печати самого итератора выводится только его ссылка.

По умолчанию к элементам применяется `operator.add`. Можно, например, указать оператор умножения:

In [9]:
import operator
list(itertools.accumulate(range(1, 10), operator.mul))

[1, 2, 6, 24, 120, 720, 5040, 40320, 362880]

# 4. Бесконечный итератор последовательности

С помощью `itertools.cycle()` создаётся кольцевой итератор. Прийдя к последнему значению, он вновь начинает с первого:

In [10]:
waltz = itertools.cycle(['и раз', 'и два', 'и три'])
next(waltz)

'и раз'

In [11]:
next(waltz)

'и два'

In [12]:
next(waltz)

'и три'

In [13]:
next(waltz)

'и раз'

# 5. Бесконечный итератор одного объекта
Итератор, создаваемый `itertools.repeat()` это вырожденный случай `itertools.cycle()`. Вместо последовательности повторяется одно и то же значение. Бесконечно или `times` раз:

In [14]:
s = "Птица Говорун отличается умом и сообразительностью"
rep = itertools.repeat(s, times=2)
next(rep)

'Птица Говорун отличается умом и сообразительностью'

In [15]:
next(rep)

'Птица Говорун отличается умом и сообразительностью'

In [16]:
next(rep)

StopIteration: 

Классический пример использования `itertools.repeat()` – итератор для `map()`:

In [17]:
nums = range(10)
squares = map(pow, nums, itertools.repeat(2))
list(squares)

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

Все числа последовательности `nums` возведены в степень `2`. Итератор вызывается столько раз, сколько необходимо, не нужно думать о числе элементов в последовательности.

# 6. Мапирование со звёздочкой – starmap

Раз мы заговорили о `map()`, полезно рассказать и о `itertools.starmap()`. Этот метод принимает функцию и список кортежей аргументов. Как если бы использовался оператор `*`, отсюда и название:

In [18]:
squares = itertools.starmap(pow, [(0, 2), (1, 2), (2, 2)])
list(squares)

[0, 1, 4]

# 7. Комбинаторика: сочетания

Модуль `itertools` позволяет решать программные задачи, построенные на структурах комбинаторики.

[Сочетания](https://ru.wikipedia.org/wiki/%D0%A1%D0%BE%D1%87%D0%B5%D1%82%D0%B0%D0%BD%D0%B8%D0%B5) – выбранные из множества `n` объектов комбинации  `m` объектов,   отличающиеся хотя бы одним объектом. Порядок элементов не важен.

Например, мы хотим составить трёхцветный флаг из лент цветных тканей. Есть четыре цвета лент. Все варианты выбора тканей без учёта их расположения:

In [19]:
colors = ['белый', 'жёлтый', 'синий', 'красный']
for item in itertools.combinations(colors, 3):
    print(item)

('белый', 'жёлтый', 'синий')
('белый', 'жёлтый', 'красный')
('белый', 'синий', 'красный')
('жёлтый', 'синий', 'красный')


Порядок следования не имеет значения, поэтому все тройки цветов уникальны.

# 8. Комбинаторика: перестановки

[Перестановки](https://ru.wikipedia.org/wiki/%D0%9F%D0%B5%D1%80%D0%B5%D1%81%D1%82%D0%B0%D0%BD%D0%BE%D0%B2%D0%BA%D0%B0) – те же сочетания, для которых важен порядок следования элементов. В продолжение предыдущего примера определим все варианты как мы можем составить флаг с учётом порядка следования цветов:

In [20]:
for item in itertools.permutations(colors, 3):
    print(item)

('белый', 'жёлтый', 'синий')
('белый', 'жёлтый', 'красный')
('белый', 'синий', 'жёлтый')
('белый', 'синий', 'красный')
('белый', 'красный', 'жёлтый')
('белый', 'красный', 'синий')
('жёлтый', 'белый', 'синий')
('жёлтый', 'белый', 'красный')
('жёлтый', 'синий', 'белый')
('жёлтый', 'синий', 'красный')
('жёлтый', 'красный', 'белый')
('жёлтый', 'красный', 'синий')
('синий', 'белый', 'жёлтый')
('синий', 'белый', 'красный')
('синий', 'жёлтый', 'белый')
('синий', 'жёлтый', 'красный')
('синий', 'красный', 'белый')
('синий', 'красный', 'жёлтый')
('красный', 'белый', 'жёлтый')
('красный', 'белый', 'синий')
('красный', 'жёлтый', 'белый')
('красный', 'жёлтый', 'синий')
('красный', 'синий', 'белый')
('красный', 'синий', 'жёлтый')


Как и ожидалось, число комбинаций с учетом порядка элементов гораздо больше.

# 9. Комбинаторика: размещение с повторениями

Размещение с повторениями (выборка с возвращением) – это комбинаторное размещение объектов, в котором каждый объект может участвовать в размещении несколько раз.

<img src="img/Пин-код.png" width="500"/>

Например, есть пин-код из четырех цифр. На каждой позиции стоит цифра от `0` до `9`. Позиции не зависят друг от друга. Переберем все возможные коды:

In [21]:
digits = range(10)
pincode_vars = itertools.product(digits, repeat=4)
for var in pincode_vars:
    print(var)

(0, 0, 0, 0)
(0, 0, 0, 1)
(0, 0, 0, 2)
(0, 0, 0, 3)
(0, 0, 0, 4)
(0, 0, 0, 5)
(0, 0, 0, 6)
(0, 0, 0, 7)
(0, 0, 0, 8)
(0, 0, 0, 9)
(0, 0, 1, 0)
(0, 0, 1, 1)
(0, 0, 1, 2)
(0, 0, 1, 3)
(0, 0, 1, 4)
(0, 0, 1, 5)
(0, 0, 1, 6)
(0, 0, 1, 7)
(0, 0, 1, 8)
(0, 0, 1, 9)
(0, 0, 2, 0)
(0, 0, 2, 1)
(0, 0, 2, 2)
(0, 0, 2, 3)
(0, 0, 2, 4)
(0, 0, 2, 5)
(0, 0, 2, 6)
(0, 0, 2, 7)
(0, 0, 2, 8)
(0, 0, 2, 9)
(0, 0, 3, 0)
(0, 0, 3, 1)
(0, 0, 3, 2)
(0, 0, 3, 3)
(0, 0, 3, 4)
(0, 0, 3, 5)
(0, 0, 3, 6)
(0, 0, 3, 7)
(0, 0, 3, 8)
(0, 0, 3, 9)
(0, 0, 4, 0)
(0, 0, 4, 1)
(0, 0, 4, 2)
(0, 0, 4, 3)
(0, 0, 4, 4)
(0, 0, 4, 5)
(0, 0, 4, 6)
(0, 0, 4, 7)
(0, 0, 4, 8)
(0, 0, 4, 9)
(0, 0, 5, 0)
(0, 0, 5, 1)
(0, 0, 5, 2)
(0, 0, 5, 3)
(0, 0, 5, 4)
(0, 0, 5, 5)
(0, 0, 5, 6)
(0, 0, 5, 7)
(0, 0, 5, 8)
(0, 0, 5, 9)
(0, 0, 6, 0)
(0, 0, 6, 1)
(0, 0, 6, 2)
(0, 0, 6, 3)
(0, 0, 6, 4)
(0, 0, 6, 5)
(0, 0, 6, 6)
(0, 0, 6, 7)
(0, 0, 6, 8)
(0, 0, 6, 9)
(0, 0, 7, 0)
(0, 0, 7, 1)
(0, 0, 7, 2)
(0, 0, 7, 3)
(0, 0, 7, 4)
(0, 0, 7, 5)
(0, 0, 7, 6)

(3, 3, 6, 0)
(3, 3, 6, 1)
(3, 3, 6, 2)
(3, 3, 6, 3)
(3, 3, 6, 4)
(3, 3, 6, 5)
(3, 3, 6, 6)
(3, 3, 6, 7)
(3, 3, 6, 8)
(3, 3, 6, 9)
(3, 3, 7, 0)
(3, 3, 7, 1)
(3, 3, 7, 2)
(3, 3, 7, 3)
(3, 3, 7, 4)
(3, 3, 7, 5)
(3, 3, 7, 6)
(3, 3, 7, 7)
(3, 3, 7, 8)
(3, 3, 7, 9)
(3, 3, 8, 0)
(3, 3, 8, 1)
(3, 3, 8, 2)
(3, 3, 8, 3)
(3, 3, 8, 4)
(3, 3, 8, 5)
(3, 3, 8, 6)
(3, 3, 8, 7)
(3, 3, 8, 8)
(3, 3, 8, 9)
(3, 3, 9, 0)
(3, 3, 9, 1)
(3, 3, 9, 2)
(3, 3, 9, 3)
(3, 3, 9, 4)
(3, 3, 9, 5)
(3, 3, 9, 6)
(3, 3, 9, 7)
(3, 3, 9, 8)
(3, 3, 9, 9)
(3, 4, 0, 0)
(3, 4, 0, 1)
(3, 4, 0, 2)
(3, 4, 0, 3)
(3, 4, 0, 4)
(3, 4, 0, 5)
(3, 4, 0, 6)
(3, 4, 0, 7)
(3, 4, 0, 8)
(3, 4, 0, 9)
(3, 4, 1, 0)
(3, 4, 1, 1)
(3, 4, 1, 2)
(3, 4, 1, 3)
(3, 4, 1, 4)
(3, 4, 1, 5)
(3, 4, 1, 6)
(3, 4, 1, 7)
(3, 4, 1, 8)
(3, 4, 1, 9)
(3, 4, 2, 0)
(3, 4, 2, 1)
(3, 4, 2, 2)
(3, 4, 2, 3)
(3, 4, 2, 4)
(3, 4, 2, 5)
(3, 4, 2, 6)
(3, 4, 2, 7)
(3, 4, 2, 8)
(3, 4, 2, 9)
(3, 4, 3, 0)
(3, 4, 3, 1)
(3, 4, 3, 2)
(3, 4, 3, 3)
(3, 4, 3, 4)
(3, 4, 3, 5)
(3, 4, 3, 6)

(7, 0, 8, 1)
(7, 0, 8, 2)
(7, 0, 8, 3)
(7, 0, 8, 4)
(7, 0, 8, 5)
(7, 0, 8, 6)
(7, 0, 8, 7)
(7, 0, 8, 8)
(7, 0, 8, 9)
(7, 0, 9, 0)
(7, 0, 9, 1)
(7, 0, 9, 2)
(7, 0, 9, 3)
(7, 0, 9, 4)
(7, 0, 9, 5)
(7, 0, 9, 6)
(7, 0, 9, 7)
(7, 0, 9, 8)
(7, 0, 9, 9)
(7, 1, 0, 0)
(7, 1, 0, 1)
(7, 1, 0, 2)
(7, 1, 0, 3)
(7, 1, 0, 4)
(7, 1, 0, 5)
(7, 1, 0, 6)
(7, 1, 0, 7)
(7, 1, 0, 8)
(7, 1, 0, 9)
(7, 1, 1, 0)
(7, 1, 1, 1)
(7, 1, 1, 2)
(7, 1, 1, 3)
(7, 1, 1, 4)
(7, 1, 1, 5)
(7, 1, 1, 6)
(7, 1, 1, 7)
(7, 1, 1, 8)
(7, 1, 1, 9)
(7, 1, 2, 0)
(7, 1, 2, 1)
(7, 1, 2, 2)
(7, 1, 2, 3)
(7, 1, 2, 4)
(7, 1, 2, 5)
(7, 1, 2, 6)
(7, 1, 2, 7)
(7, 1, 2, 8)
(7, 1, 2, 9)
(7, 1, 3, 0)
(7, 1, 3, 1)
(7, 1, 3, 2)
(7, 1, 3, 3)
(7, 1, 3, 4)
(7, 1, 3, 5)
(7, 1, 3, 6)
(7, 1, 3, 7)
(7, 1, 3, 8)
(7, 1, 3, 9)
(7, 1, 4, 0)
(7, 1, 4, 1)
(7, 1, 4, 2)
(7, 1, 4, 3)
(7, 1, 4, 4)
(7, 1, 4, 5)
(7, 1, 4, 6)
(7, 1, 4, 7)
(7, 1, 4, 8)
(7, 1, 4, 9)
(7, 1, 5, 0)
(7, 1, 5, 1)
(7, 1, 5, 2)
(7, 1, 5, 3)
(7, 1, 5, 4)
(7, 1, 5, 5)
(7, 1, 5, 6)
(7, 1, 5, 7)

# 10. Комбинаторика: размещение

Рассмотрим также случай обычного размещения, когда  элементы могут повторяться, но каждое сочетание встречается только один раз:

In [22]:
letters = 'ABCD'

code_vars = itertools.combinations_with_replacement(letters, 2)
for var in code_vars:
    print(var)

('A', 'A')
('A', 'B')
('A', 'C')
('A', 'D')
('B', 'B')
('B', 'C')
('B', 'D')
('C', 'C')
('C', 'D')
('D', 'D')


# 11. Декартово произведение множеств

Метод `itertools.product()` можно использовать не только для размещений с повторениями. 

Декартово (прямое) произведение – множество, элементами которого являются все возможные упорядоченные пары элементов исходных множеств.

<img src="img/Шахматы.jpg" width="500"/>

Например, найдём обозначения всех полей шахматной доски:

In [23]:
import string
letters = list(string.ascii_lowercase[:8])
digits = range(1, 9)
for (letter, digit) in itertools.product(letters, digits):
    print(letter+str(digit), end=' ')

a1 a2 a3 a4 a5 a6 a7 a8 b1 b2 b3 b4 b5 b6 b7 b8 c1 c2 c3 c4 c5 c6 c7 c8 d1 d2 d3 d4 d5 d6 d7 d8 e1 e2 e3 e4 e5 e6 e7 e8 f1 f2 f3 f4 f5 f6 f7 f8 g1 g2 g3 g4 g5 g6 g7 g8 h1 h2 h3 h4 h5 h6 h7 h8 

# 12. Цепочки итераторов

Иногда необходимо использовать нескольков итераторов. И независимо, и цепочкой один за другим. Для объединения итераторов используйте `itertools.chain(*iterables)`.

<img src="img/Карты.jpg" width="500"/>

Например, мы хотим использовать для отрисовки игральных карт независимые итераторы обозначений в углу поля карты:

In [24]:
num_cards = [str(i) for i in range(2, 11)]
face_cards  = ['В', 'Д', 'К', 'Т']

list(itertools.chain(num_cards, face_cards))

['2', '3', '4', '5', '6', '7', '8', '9', '10', 'В', 'Д', 'К', 'Т']

С помощью `itertools.chain()` также можно добавлять отдельные элементы в начало итератора:

In [25]:
def prepend(value, iterator):
    return itertools.chain([value], iterator)

list(prepend(1, [2, 3, 4]))

[1, 2, 3, 4]

# 13. Плоский список из вложенного

Альтернативным конструктором `itertools.chain()` служит `itertools.chain.from_iterable()`. Метод принимает один итерируемый объект. Сравните их вызовы:

In [26]:
list(itertools.chain('ABC', 'DEF'))

['A', 'B', 'C', 'D', 'E', 'F']

In [27]:
list(itertools.chain.from_iterable(['ABC', 'DEF']))

['A', 'B', 'C', 'D', 'E', 'F']

Последний конструктор удобно использовать для объединения списков:

In [28]:
list_of_lists = [[1, 2], [3, 4, 5], [6, 7, 8, 9]]

list(itertools.chain.from_iterable(list_of_lists))

[1, 2, 3, 4, 5, 6, 7, 8, 9]

# 14. Итератор среза

Срез – удобный инструмент списков, который доступен и для итераторов с помощью `itertools.islice()`.

Например, нам достаточно читать из крупного файла только три первых строки:

In [29]:
with open('test.txt', 'r') as f:
    header = itertools.islice(f, 3)
    
    for line in header:
        print(line, end = '')

Строка 1
Строка 2
Строка 3


Функция `itertools.islice()` позволяет итерироваться по любым объектам в формате среза. Например, следующая функция возвращает `n` первых элементов итерируемого объекта в виде списка:

In [30]:
def take(n, iterable):
    return list(islice(iterable, n))

# 15. Фильтрация группы элементов

Функция `compress()` оставляет из итерируемых данных только те, что соответствуют позициям булевых селекторов:

In [31]:
numbers = [0, 1, 2, 3, 2, 1, 0]
selectors = [True, True, False, True]

list(itertools.compress(numbers, selectors))

[0, 1, 3]

Метод `itertools.filterfalse()` дополняет обычный фильтр `filter()`:

In [32]:
def filter_func(n):
    if n < 2:
        return True
    return False

print(list(filter(filter_func, numbers)))
print(list(itertools.filterfalse(filter_func, numbers)))

[0, 1, 1, 0]
[2, 3, 2]


# 16. Фильтрация до последнего истинного (или с первого ложного) элемента

Если необходимо отобрать объекты, стоящие после неудовлетворяющего условию элемента, используем `itertools.dropwhile()`:

In [33]:
list(itertools.dropwhile(filter_func, numbers))

[2, 3, 2, 1, 0]

Метод `itertools.takewhile()` наборот выведет элементы, удовлетворяющие условию вплоть до объекта, прерывающего цепочку истинных элементов:

In [34]:
list(itertools.takewhile(filter_func, numbers))

[0, 1]

# 17. Группировка по ключу

<img src="img/Ключ.png" width="500"/>

Инструмент `itertools.groupby()` объединяет смежные словари в группы по общему ключу. Например, сгруппируем студентов с одинаковой оценкой:

In [35]:
people = [{"Имя": "Петр",
           "Отчество": "Петрович",
           "Фамилия": "Петров",
           "Оценка":5},
          {"Имя": "Ольга",
           "Отчество": "Алексеевна",
           "Фамилия": "Иванова",
           "Оценка":5},
          {"Имя": "Николай",
           "Отчество": "Николаевич",
           "Фамилия": "Николаев",
           "Оценка":4},
          {"Имя": "Федор",
           "Отчество": "Владимирович",
           "Фамилия": "Иванов",
           "Оценка":3},
          {"Имя": "Владимир",
           "Отчество": "Федорович",
           "Фамилия": "Иванов",
           "Оценка":3}]

def get_mark(person):
    return person['Оценка']

person_marks = itertools.groupby(people, get_mark)

for key, group in person_marks:
    print(key)
    for person in group:
        print(person)
    print()

5
{'Имя': 'Петр', 'Отчество': 'Петрович', 'Фамилия': 'Петров', 'Оценка': 5}
{'Имя': 'Ольга', 'Отчество': 'Алексеевна', 'Фамилия': 'Иванова', 'Оценка': 5}

4
{'Имя': 'Николай', 'Отчество': 'Николаевич', 'Фамилия': 'Николаев', 'Оценка': 4}

3
{'Имя': 'Федор', 'Отчество': 'Владимирович', 'Фамилия': 'Иванов', 'Оценка': 3}
{'Имя': 'Владимир', 'Отчество': 'Федорович', 'Фамилия': 'Иванов', 'Оценка': 3}



Обратите внимание, что группировка работает только со смежными объектами. Предварительно отсортируйте данные.

# 18. Репликация итераторов

Функция `itertools.tee()` создаёт из одного итерируемого объекта два итератора:

In [36]:
letters = 'abc'
it1, it2 = itertools.tee(letters)
next(it1)

'a'

In [37]:
next(it1)

'b'

In [38]:
next(it2)

'a'

Эти итераторы соответствуют одной последовательности, но независимы друг от друга.

# 19. Повторение последовательности заданное число раз

Волшебная сила itertools – в умении комбинировать итераторы, чтобы писать быстрый, эффективный и ясный код.

Например, сочетание `itertools.chain()` и `itertools.from_iterable()` даёт ограниченный вариант бесконечного `itertools.cycle()`:

In [39]:
def ncycles(iterable, n):
    return itertools.chain(itertools.from_iterable(repeat(tuple(iterable), n)))

# 20. Уникальные элементы последовательности

Напоследок напишем функцию, выводящую все уникальные элементы в том же порядке, как они появлялись в исходной последовательности. Опциональный аргумент – ссылка на функцию определения эквивалентов, представленных по-разному. Например, заглавных и строчных букв.

In [40]:
def unique_everseen(iterable, key=None):
    seen = set()
    seen_add = seen.add
    if key is None:
        for element in itertools.filterfalse(seen.__contains__, iterable):
            seen_add(element)
            yield element
    else:
        for element in iterable:
            k = key(element)
            if k not in seen:
                seen_add(k)
                yield element

In [41]:
list(unique_everseen('Абракадааааабра'))

['А', 'б', 'р', 'а', 'к', 'д']

In [42]:
list(unique_everseen('Абракадааааабра', str.lower))

['А', 'б', 'р', 'к', 'д']

# Заключение

Описывая приёмы использования itertools, мы попутно определили основные функции модуля.

Итераторы полезны для обработки крупных файлов и потоков данных, для доступа к содержимому объектов без раскрытия их полного внутреннего представления.

Модуль itertools обеспечивает ключевые структуры итераторов Python. Другие шаблоны вы найдёте в специальной библиотеке примеров `more-itertools`:

`pip install more-itertools`